In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

#  read the dataset
df = pd.read_csv("2012-sat-results.csv")

print(df.info())
print("")

# convert all values to numeric
df["SAT Critical Reading Avg. Score"] = pd.to_numeric(df["SAT Critical Reading Avg. Score"], errors="coerce")
df["SAT Math Avg. Score"] = pd.to_numeric(df["SAT Math Avg. Score"], errors="coerce")
df["SAT Writing Avg. Score"] = pd.to_numeric(df["SAT Writing Avg. Score"], errors="coerce")

# Drop rows with NaN values
df = df.dropna(subset=["SAT Critical Reading Avg. Score", "SAT Math Avg. Score", "SAT Writing Avg. Score"])

print(df.info())
print("")

# population params
mu = df["SAT Writing Avg. Score"].mean()
tao = df["SAT Writing Avg. Score"].sum()
sigmasq = df["SAT Writing Avg. Score"].var(ddof=0)

print(f"The mu is: {mu}")
print(f"The tao is: {tao}")
print(f"The sigma^2 is: {sigmasq}")

print("")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   DBN                              478 non-null    object
 1   SCHOOL NAME                      478 non-null    object
 2   Num of SAT Test Takers           478 non-null    object
 3   SAT Critical Reading Avg. Score  478 non-null    object
 4   SAT Math Avg. Score              478 non-null    object
 5   SAT Writing Avg. Score           478 non-null    object
dtypes: object(6)
memory usage: 22.5+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 421 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              421 non-null    object 
 1   SCHOOL NAME                      421 non-null    object 
 2   Num of SA

#### M = 421
#### ${\mu}$ = 393.9857
#### sample_size = 80

### Problem 1: Divide your population into N (3 to 5) primary units with different number of secondary units (Nh should not be equal).

In [4]:
# N = 5 groups

bins = [280, 370, 410, 460, 500, df["SAT Critical Reading Avg. Score"].max()]
labels = [0, 1, 2, 3, 4]

# Create the Reading_Stratum column
df["Reading_Stratum"] = pd.cut(df["SAT Critical Reading Avg. Score"], bins=bins, labels=labels, include_lowest=True)

# Calculate Nh and sigma_sq_h
reading = {
    "Nh": df.groupby("Reading_Stratum", observed=False).size().tolist(),
    "sigma_sq_h": df.groupby("Reading_Stratum", observed=False)["SAT Critical Reading Avg. Score"].var(ddof=1).tolist()
}
reading

{'Nh': [120, 178, 75, 23, 24],
 'sigma_sq_h': [359.5436974789913,
  125.25550688757683,
  192.81981981981997,
  153.30039525691697,
  2583.644927536234]}

# Problem 2: Perform two stage design with SRS at each stage

In [6]:
np.random.seed(422)

N = 5
n = 4
sample_size = 80

primary_grps = np.random.choice(labels, size=n, replace=False)
primary_grps

array([0, 1, 4, 3])

In [7]:
df_selected = df[df["Reading_Stratum"].isin(primary_grps)]

M_i = df_selected.groupby("Reading_Stratum", observed=False).size()

# mi proportional to size of each primary unit
m_i = (M_i / M_i.sum() * sample_size).round().astype(int)

# Make sure that m_i is exactly 80
while m_i.sum() != sample_size:
    diff = sample_size - m_i.sum()
    idx = np.random.choice(m_i.index)
    m_i[idx] += diff

sampled_df_list = []

# Sample mi[i] units for primary_group[i]
for group, num_samples in m_i.items():
    group_df = df_selected[df_selected["Reading_Stratum"] == group]
    sampled_group = group_df.sample(n=num_samples, random_state=5577) #70 #5577
    sampled_group["Selected_Primary_Group"] = group
    sampled_df_list.append(sampled_group)

final_sampled_df = pd.concat(sampled_df_list)

In [8]:
final_sampled_df.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Reading_Stratum,Selected_Primary_Group
291,16K393,FREDERICK DOUGLASS ACADEMY IV SECONDARY SCHOOL,20,355.0,355.0,358.0,0,0
383,24Q530,INTERNATIONAL HIGH SCHOOL AT LAGUARDIA COMMUNI...,69,326.0,409.0,329.0,0,0
170,09X403,BRONX INTERNATIONAL HIGH SCHOOL,49,314.0,312.0,339.0,0,0
300,17K524,INTERNATIONAL HIGH SCHOOL AT PROSPECT HEIGHTS,71,287.0,335.0,291.0,0,0
115,06M467,HIGH SCHOOL FOR LAW AND PUBLIC SERVICE,94,363.0,378.0,361.0,0,0


## Problem 3: Estimate your parameter of interest by unbiased estimator. Estimate its variance and standard deviation.

$\Large\hat{y_i} = \frac{M_i}{m_i} \sum_{j=1}^{m_i}y_{ij} = M_i \bar{y_i}$

$\Large\bar{y_i} = \frac{1}{m_i} \sum_{j=1}^{m_i}y_{ij} = \frac{\hat{y_i}}{M_i}$

In [11]:
ybar_i = final_sampled_df.groupby("Selected_Primary_Group")["SAT Critical Reading Avg. Score"].mean()

Mi_selected = M_i.loc[ybar_i.index]
mi_selected = m_i.loc[ybar_i.index]
yhat_i = Mi_selected * ybar_i

In [12]:
yhat_i

Selected_Primary_Group
0    41091.428571
1    69003.219512
3    10819.200000
4    12944.000000
dtype: float64

In [13]:
ybar_i

Selected_Primary_Group
0    342.428571
1    387.658537
3    470.400000
4    539.333333
Name: SAT Critical Reading Avg. Score, dtype: float64

$\Large\hat{\tau} = \frac{N}{n} \sum_{i=1}^n$

$\Large\hat{\mu} = \frac{\hat{\tau}}{M}$

In [15]:
M = 421
tau_hat = (N / n) * sum(yhat_i)
mu_hat = tau_hat / M
mu_hat

397.4401665190727

$\Large s^2_u = \frac{1}{n-1}\sum_{i=1}^n (\hat{y_i} - \hat{\mu_{1}})^2$

$\mu_1 = \frac{1}{n}\sum_{i=1}^n \hat{y_i}$

$\Large s^2_i = \frac{1}{m_i} \sum_{j=1}^{m_i}(y_{ij} - \bar{y_i})^2$

$\Large\hat{\text{var}}(\hat{\tau}) = N(N-n)\frac{s^2_u}{n} + \frac{N}{n}\sum_{i=1}^n M_i(M_i - m_i) \frac{s^2_i}{m_i}$

$\Large\hat{\text{var}}(\hat{\mu}) = \frac{\hat{\text{var}}(\hat{\tau})}{M^2}$

In [17]:
M = 421
mu_hat_1 = 1/n * sum(yhat_i)

su_sq = ((yhat_i - mu_hat_1)**2).sum() / (n - 1)
si_sq = final_sampled_df.groupby("Selected_Primary_Group")["SAT Critical Reading Avg. Score"].var(ddof=1)
var_hat_tau_hat = N*(N-n)*su_sq/n + (N/n)*(Mi_selected*(Mi_selected - mi_selected)*si_sq/mi_selected).sum()
var_hat_mu_hat = var_hat_tau_hat / M**2

In [18]:
var_hat_mu_hat

5303.92544410031

Estimated sd: sqrt($\hat{\text{var}}(\hat{\mu})$)

In [20]:
sd_mu_hat = var_hat_mu_hat**0.5
sd_mu_hat

72.82805396343026

$\hat{\mu}$: 397.4401665190727

Estimated Variance: 5303.92544410031

Estimated standard deviation: 72.82805396343026

# 4. Estimate your parameter of interest by ratio estimator. Estimate its variance and standard deviation

In [23]:
ybar_i = final_sampled_df.groupby("Selected_Primary_Group")["SAT Critical Reading Avg. Score"].mean()

Mi_selected = M_i.loc[ybar_i.index]
mi_selected = m_i.loc[ybar_i.index]
yhat_i = Mi_selected * ybar_i

M = 421

$\Large\hat{\mu_r} = \hat{r} = \frac{\sum_{i=1}^n \hat{y_i}}{\sum_{i=1}^n M_i}$

In [25]:
r_hat = sum(yhat_i) / sum(Mi_selected)

mu_hat_r = r_hat
mu_hat_r

387.9937625612281

$\Large\hat{\text{var}}(\hat{\tau_r}) = \frac{N(N-n)}{n(n-1)} \sum_{i=1}^n(\hat{y_i} - M_i\hat{r})^2 + \frac{N}{n}\sum_{i=1}^n M_i(M_i - m_i)\frac{s^2_i}{m_i}$

$\Large\hat{\text{var}}(\hat{\mu_r}) = \frac{\Large\hat{\text{var}}(\hat{\tau_r})}{M^2}$

In [27]:
var_hat_tau_hat_r = ((N * (N - n)) /  (n * (n - 1))) * sum( (yhat_i - Mi_selected * r_hat)**2 ) + (N / n) * sum( (Mi_selected * (Mi_selected - mi_selected)) * (si_sq / mi_selected))
var_hat_mu_hat_r = var_hat_tau_hat_r / (M**2)
var_hat_mu_hat_r

112.34620577465252

Estimated standard deviation: sqrt($\hat{\text{var}}(\hat{\mu}_r)$)

In [29]:
sd_mu_hat_r = var_hat_mu_hat_r**0.5
sd_mu_hat_r

10.599349309021404

$\hat{\mu}_r$: 387.9937

Estimated Variance: 112.3462

Estimated standard deviation: 10.5993

# 5. Perform two stage design in which primary units are selected with replacement, with probabilities proportional to size and a sample of secondary units is selected independently using SRS from the selected primary units on the first stage with the total size $\sum^n_{i=1} m_i$ chosen in Report 2

In [32]:
M_i = df.groupby("Reading_Stratum", observed=False).size()

sample_size = 80
n_primary_draws = 10
p_i = M_i / M_i.sum()

# select primaries w/ replacement w/ PPS
primary_units_selected = np.random.choice(M_i.index, size=n_primary_draws, replace=True, p=p_i)
primary_counts = pd.Series(primary_units_selected).value_counts()

m_i = (primary_counts / primary_counts.sum() * sample_size).round().astype(int)

# Make sure that m_i is exactly 80
while m_i.sum() != sample_size:
    diff = sample_size - m_i.sum()
    idx = np.random.choice(m_i.index)
    m_i[idx] += diff

sampled_df_list = []
# Sample mi[i] units for primary_group[i]
for group, num_samples in m_i.items():
    group_df = df[df["Reading_Stratum"] == group]

    # If more samples are needed than exist in this primary group
    sampled_group = group_df.sample(n=num_samples, random_state=5577)

    sampled_group["Selected_Primary_Group"] = group
    sampled_df_list.append(sampled_group)

# Combine all sampled secondary units
final_sampled_df = pd.concat(sampled_df_list)
final_sampled_df.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Reading_Stratum,Selected_Primary_Group
200,10X477,MARBLE HILL HIGH SCHOOL FOR INTERNATIONAL STUDIES,99,414.0,435.0,414.0,2,2
361,21K690,BROOKLYN STUDIO SECONDARY SCHOOL,119,429.0,449.0,435.0,2,2
54,02M531,REPERTORY COMPANY HIGH SCHOOL FOR THEATRE ARTS,42,429.0,404.0,420.0,2,2
313,17K600,CLARA BARTON HIGH SCHOOL,259,425.0,413.0,413.0,2,2
79,03M404,INNOVATION DIPLOMA PLUS,12,416.0,403.0,381.0,2,2


# 6. Estimate your parameter of interest by Hansen-Horvitz estimator. Estimate its variance and standard deviation.

Hansen-Horvitz Estimator: 
- $\hat{\mu}_p = \frac{\hat{\tau}_p}{M}$ 
- $\hat{\tau}_p = \frac{M}{n} \sum_{i=1}^{n} \bar{y}_i$
- $\bar{y}_i = \frac{1}{m_i} \sum_{j=1}^{m_i} y_{ij}
\quad \text{(sample mean within the \(i\)-th primary unit)}$


In [35]:
y_i_bar = final_sampled_df.groupby("Selected_Primary_Group")["SAT Critical Reading Avg. Score"].mean()
tau_p_hat = (M/n) * sum(y_i_bar)
mu_p_hat = tau_p_hat / M
print(f"Estimated Mean (mu_p_hat): {mu_p_hat}")

Estimated Mean (mu_p_hat): 427.8333333333333


Estimated Variance:
- $\hat{\text{var}}(\hat{\mu_p})$ = $\frac{1}{n(n-1)} \sum_{i=1}^{n} \left( \bar{y}_i - \hat{\mu}_p \right)^2$


In [37]:
var_hat_mu_p_hat = (1 / (n * (n-1))) * sum((y_i_bar - mu_p_hat)**2)
print(f"Estimated Variance: {var_hat_mu_p_hat}")

Estimated Variance: 1949.4029947916665


Estimated Standard Deviation:
- std. dev = sqrt($\hat{\text{var}}(\hat{\mu}_p)$)

In [39]:
print(f"Estimated Standard Deviation: {np.sqrt(var_hat_mu_p_hat)}")

Estimated Standard Deviation: 44.15204406130781


# Question 7

Divide your population into N primary units in a different way from part 1. Again, Nh should not be equal. You can use any variable from the dataset. It does not have to be one of the two that were used as auxiliary variables or for stratification.

In [42]:
math_bins = [280, 370, 410, 460, 500, df["SAT Math Avg. Score"].max()]
math_labels = [0, 1, 2, 3, 4]

df["Math_Stratum"] = pd.cut(df["SAT Math Avg. Score"], bins=math_bins, labels=math_labels, include_lowest=True)

math = {
    "Nh": df.groupby("Math_Stratum", observed=False).size().tolist(),
    "sigma_sq_h_math": df.groupby("Math_Stratum", observed=False)["SAT Math Avg. Score"].var(ddof=1).tolist()
}
math

{'Nh': [98, 158, 93, 38, 34],
 'sigma_sq_h_math': [208.43183252682488,
  118.64161896315369,
  189.3796166432913,
  126.38762446657215,
  3576.9919786096248]}

### 8. Repeat steps 2-6.

In [44]:
np.random.seed(4)

N = 5
n = 4
sample_size = 80

primary_grps = np.random.choice(labels, size=n, replace=False)
primary_grps

array([0, 3, 1, 4])

In [45]:
df_selected = df[df["Math_Stratum"].isin(primary_grps)]

M_i = df_selected.groupby("Math_Stratum", observed=False).size()

# mi proportional to size of each primary unit
m_i = (M_i / M_i.sum() * sample_size).round().astype(int)

# Make sure that m_i is exactly 80
while m_i.sum() != sample_size:
    diff = sample_size - m_i.sum()
    idx = np.random.choice(m_i.index)
    m_i[idx] += diff

sampled_df_list = []

# Sample mi[i] units for primary_group[i]
for group, num_samples in m_i.items():
    group_df = df_selected[df_selected["Math_Stratum"] == group]
    sampled_group = group_df.sample(n=num_samples, random_state=11445) #70 #5577 #18111 #9931
    sampled_group["Selected_Primary_Group"] = group
    sampled_df_list.append(sampled_group)

final_sampled_df = pd.concat(sampled_df_list)
final_sampled_df.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Reading_Stratum,Math_Stratum,Selected_Primary_Group
227,12X248,"METROPOLITAN HIGH SCHOOL, THE",49,347.0,367.0,341.0,0,0,0
240,12X682,FANNIE LOU HAMER FREEDOM HIGH SCHOOL,72,342.0,346.0,341.0,0,0,0
461,32K403,ACADEMY FOR ENVIRONMENTAL LEADERSHIP,50,380.0,366.0,352.0,1,0,0
106,05M685,BREAD & ROSES INTEGRATED ARTS HIGH SCHOOL,61,369.0,361.0,355.0,0,0,0
156,08X650,JANE ADDAMS HIGH SCHOOL FOR ACADEMIC CAREERS,78,377.0,359.0,376.0,1,0,0


In [46]:
ybar_i = final_sampled_df.groupby("Selected_Primary_Group")["SAT Math Avg. Score"].mean()

Mi_selected = M_i.loc[ybar_i.index]
mi_selected = m_i.loc[ybar_i.index]
yhat_i = Mi_selected * ybar_i
yhat_i

Selected_Primary_Group
0    34643.000000
1    61036.615385
3    18159.777778
4    20106.750000
dtype: float64

In [47]:
ybar_i

Selected_Primary_Group
0    353.500000
1    386.307692
3    477.888889
4    591.375000
Name: SAT Math Avg. Score, dtype: float64

In [48]:
M = 421
tau_hat = (N / n) * sum(yhat_i)
mu_hat = tau_hat / M
mu_hat

397.7023253040177

In [49]:
M = 421
mu_hat_1 = 1/n * sum(yhat_i)

su_sq = ((yhat_i - mu_hat_1)**2).sum() / (n - 1)
si_sq = final_sampled_df.groupby("Selected_Primary_Group")["SAT Math Avg. Score"].var(ddof=1)
var_hat_tau_hat = N*(N-n)*su_sq/n + (N/n)*(Mi_selected*(Mi_selected - mi_selected)*si_sq/mi_selected).sum()
var_hat_mu_hat = var_hat_tau_hat / M**2
var_hat_mu_hat


2765.764591069349

In [50]:
sd_mu_hat = var_hat_mu_hat**0.5
sd_mu_hat

52.590537086716935

In [51]:
ybar_i = final_sampled_df.groupby("Selected_Primary_Group")["SAT Math Avg. Score"].mean()

Mi_selected = M_i.loc[ybar_i.index]
mi_selected = m_i.loc[ybar_i.index]
yhat_i = Mi_selected * ybar_i

M = 421
r_hat = sum(yhat_i) / sum(Mi_selected)

mu_hat_r = r_hat
mu_hat_r

408.37238769022304

In [52]:
var_hat_tau_hat_r = ((N * (N - n)) /  (n * (n - 1))) * sum( (yhat_i - Mi_selected * r_hat)**2 ) + (N / n) * sum( (Mi_selected * (Mi_selected - mi_selected)) * (si_sq / mi_selected))
var_hat_mu_hat_r = var_hat_tau_hat_r / (M**2)
var_hat_mu_hat_r

209.19682271278216

In [53]:
sd_mu_hat_r = var_hat_mu_hat_r**0.5
sd_mu_hat_r

14.463637948759025

In [54]:
np.random.seed(14151) #6825

M_i = df.groupby("Math_Stratum", observed=False).size()
N = 5
n = 4
sample_size = 80
n_primary_draws = 10
p_i = M_i / M_i.sum()

# select primaries w/ replacement w/ PPS
primary_units_selected = np.random.choice(M_i.index, size=n_primary_draws, replace=True, p=p_i)
primary_counts = pd.Series(primary_units_selected).value_counts()

m_i = (primary_counts / primary_counts.sum() * sample_size).round().astype(int)

# Make sure that m_i is exactly 80
while m_i.sum() != sample_size:
    diff = sample_size - m_i.sum()
    idx = np.random.choice(m_i.index)
    m_i[idx] += diff

sampled_df_list = []
# Sample mi[i] units for primary_group[i]
for group, num_samples in m_i.items():
    group_df = df[df["Math_Stratum"] == group]

    sampled_group = group_df.sample(n=num_samples, random_state=14151) #11445 19687

    sampled_group["Selected_Primary_Group"] = group
    sampled_df_list.append(sampled_group)

# Combine all sampled secondary units
final_sampled_df = pd.concat(sampled_df_list)
final_sampled_df.head()


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Reading_Stratum,Math_Stratum,Selected_Primary_Group
223,11X542,PELHAM PREPARATORY ACADEMY,76,431.0,438.0,419.0,2,2,2
374,24Q264,ACADEMY OF FINANCE AND ENTERPRISE,89,405.0,454.0,421.0,1,2,2
385,24Q560,"ROBERT F. WAGNER, JR. SECONDARY SCHOOL FOR ART...",80,443.0,447.0,440.0,2,2,2
196,10X440,DEWITT CLINTON HIGH SCHOOL,443,419.0,426.0,410.0,2,2,2
390,25Q263,FLUSHING INTERNATIONAL HIGH SCHOOL,45,314.0,417.0,318.0,0,2,2


In [55]:
y_i_bar = final_sampled_df.groupby("Selected_Primary_Group")["SAT Math Avg. Score"].mean()
tau_p_hat = (M/n) * sum(y_i_bar)
mu_p_hat = tau_p_hat / M
print(f"Estimated Mean (mu_p_hat): {mu_p_hat}")

Estimated Mean (mu_p_hat): 413.421875


In [56]:
var_hat_mu_p_hat = (1 / (n * (n-1))) * sum((y_i_bar - mu_p_hat)**2)
print(f"Estimated Variance: {var_hat_mu_p_hat}")
print(f"Estimated Standard Deviation: {np.sqrt(var_hat_mu_p_hat)}")


Estimated Variance: 564.662353515625
Estimated Standard Deviation: 23.76262513939958


### 9. Choose the best estimator of your parameter based on the smallest standard deviation (variance)

The best estimator is the ratio estimator because it has the smallest standard deviation: 14.4636 (math), 10.5993 (reading)